In [ ]:
pip install pydicom

In [ ]:
import os
import numpy as np
import pydicom
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

print("Libraries imported successfully.")

In [ ]:
def load_dicom_data(root_folder):
    dicom_data = []

    for root, _, files in os.walk(root_folder):
        for file in files:
            if file.endswith(".dcm"):
                file_path = os.path.join(root, file)
                dicom = pydicom.dcmread(file_path)
                dicom_array = dicom.pixel_array

                if len(dicom_data) == 0 or dicom_array.shape == dicom_data[0].shape:
                    dicom_data.append(dicom_array)
                else:
                    print(f"Ignoring file {file} due to mismatched dimensions.")

    return np.array(dicom_data)

data_folder = '/content/drive/MyDrive/Data'
dicom_data = load_dicom_data(data_folder)

print(f"Loaded {len(dicom_data)} DICOM files.")

In [ ]:
labels = np.random.randint(2, size=len(dicom_data))

train_data, test_data, train_labels, test_labels = train_test_split(
    dicom_data, labels, test_size=0.2, random_state=42
)

train_data = train_data / 255.0
test_data = test_data / 255.0

print("Data prepared successfully.")

In [ ]:
print("Shape of dicom_data:", dicom_data.shape)
print("Shape of train_data:", train_data.shape)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(train_data.shape[1], train_data.shape[2])),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

print("Model built successfully.")

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

train_data_tensor = tf.convert_to_tensor(train_data, dtype=tf.float32)
train_labels_tensor = tf.convert_to_tensor(train_labels, dtype=tf.float32)
test_data_tensor = tf.convert_to_tensor(test_data, dtype=tf.float32)
test_labels_tensor = tf.convert_to_tensor(test_labels, dtype=tf.float32)

print("Data converted to tensors successfully.")

In [ ]:
num_epochs = 10
history = model.fit(train_data_tensor, train_labels_tensor, epochs=num_epochs, validation_data=(test_data_tensor, test_labels_tensor))

print("Model trained successfully.")

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

print("Results visualized.")